In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/dataset'
!git clone https://github.com/backseason/SCNet.git


fatal: destination path 'SCNet' already exists and is not an empty directory.


In [ ]:
%cd SCNet


/content/SCNet


In [ ]:
from scnet import scnet50


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Import 'scnet50' from Google Drive
import sys
sys.path.append('/content/drive/MyDrive')

from scnet import scnet50

# Continue with the rest of your code...

# Define the model
model = scnet50(pretrained=False)  # Assuming you're not using a pre-trained model
# Modify the last layer for binary classification (with_mask, without_mask)
model.fc = nn.Linear(model.fc.in_features, 2)

# Set up data transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset from Google Drive
dataset_path = '/content/drive/MyDrive/dataset'  # Update the path
train_dataset = ImageFolder(dataset_path, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    # Training
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 0:  # Print every 10 batches
            print(f'  Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}')

    average_train_loss = running_loss / len(train_loader)
    print(f'Training Loss: {average_train_loss:.4f}')

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for i, (inputs, labels) in enumerate(val_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            if i % 10 == 0:  # Print every 10 batches
                print(f'  Validation Batch {i}/{len(val_loader)}')

        average_val_loss = val_loss / len(val_loader)
        accuracy = correct / total
        print(f'Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/path/to/mask_detection_model.pth')
print('Model saved.')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/2
  Batch 10/97, Loss: 1.1847
  Batch 20/97, Loss: 0.4528
  Batch 30/97, Loss: 0.3626
  Batch 40/97, Loss: 0.3535
  Batch 50/97, Loss: 0.3078


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  Batch 60/97, Loss: 0.4472
  Batch 70/97, Loss: 0.3084
  Batch 80/97, Loss: 0.3951
  Batch 90/97, Loss: 0.4027
Training Loss: 0.5382
  Validation Batch 10/25
  Validation Batch 20/25
Validation Loss: 0.4294, Accuracy: 78.65%
Epoch 2/2
